### Import bibliotek

In [23]:
import requests
from bs4 import BeautifulSoup
import json
import os

### Narzędzia

In [ ]:
def extract(ancestor, selector=None, attribute=None, many=False):
    if selector:
        if many:
            if attribute:
                return [item[attribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None

    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()


review_scheme = {
    "review_id": (None, "data-entry-id"),
    "author": (".user-post__author-name",),
    "content": (".user-post__text",),
    "score": (".user-post__score .user-post__score-count",),
    "recomendation": (".user-post__author-recomendation > em",),
    "pros": (".review-feature__item.review-feature__item--positive", None, True),
    "cons": (".review-feature__item.review-feature__item--negative", None, True),
    "likes": (".vote-yes > span",),
    "dislikes": (".vote-no > span",),
    "publish_date": (".user-post__published time:nth-of-type(1)", "datetime"),
    "purchase_date": (".user-post__published time:nth-of-type(2)", "datetime"),
}

### Pobranie ze strony Ceneo.pl opinii o konkretnym produkcie

In [25]:


with open("./headers.json", "r") as file:
    headers = json.load(file)


In [26]:
# product_id = "84514582"
product_id = input('Podaj id produktu')
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    res = requests.get(next_page, headers=headers)
    print(res.status_code)
    print(next_page)
    if res.status_code == 200:

        page_dom = BeautifulSoup(res.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")

        for review in reviews:
            single_review = {
                key: extract(review, *value) for key, value in review_scheme.items()
            }
            all_reviews.append(single_review)

        print(len(all_reviews))

        try:
            next_page = "https://www.ceneo.pl" + extract(
                page_dom, ".pagination__next", "href"
            )
        except TypeError:
            next_page = None

200
https://www.ceneo.pl/96964384#tab=reviews
10
200
https://www.ceneo.pl/96964384/opinie-2
20
200
https://www.ceneo.pl/96964384/opinie-3
30
200
https://www.ceneo.pl/96964384/opinie-4
40
200
https://www.ceneo.pl/96964384/opinie-5
50
200
https://www.ceneo.pl/96964384/opinie-6
60
200
https://www.ceneo.pl/96964384/opinie-7
70
200
https://www.ceneo.pl/96964384/opinie-8
80
200
https://www.ceneo.pl/96964384/opinie-9
90
200
https://www.ceneo.pl/96964384/opinie-10
98


### Zapisanie wszystkich opinii o konkretnym produkcie w bazie danych


In [27]:
if not os.path.exists('./opinions'):
    os.mkdir('./opinions')

In [28]:
with open(f'./opinions/{product_id}.json', 'w', encoding="UTF-8") as file:
    json.dump(all_reviews, file, indent=4,ensure_ascii=False)